# Prep

In [1]:
import pandas as pd

import datetime as dt
import os
import io
from github import Github
import logging

import Universes
import importlib # for not reloading the utils every time after adjustment
import utils_for_alpaca

In [6]:

import alpaca
from alpaca.common.enums import Sort
from alpaca.trading.enums import OrderSide, TimeInForce, AssetClass, AssetStatus, AssetExchange, OrderStatus, OrderType, OrderClass, QueryOrderStatus, CorporateActionType, CorporateActionSubType
from alpaca.trading.requests import GetCalendarRequest, GetAssetsRequest, GetOrdersRequest, MarketOrderRequest, LimitOrderRequest, StopLossRequest, TakeProfitRequest, TrailingStopOrderRequest, ReplaceOrderRequest, GetPortfolioHistoryRequest, GetCorporateAnnouncementsRequest
from alpaca.data.requests import StockLatestQuoteRequest, StockTradesRequest, StockLatestTradeRequest, StockQuotesRequest, StockBarsRequest, StockSnapshotRequest, StockLatestBarRequest, CorporateActionsRequest
from alpaca.data.timeframe import TimeFrame, TimeFrameUnit
from alpaca.data.enums import Adjustment, DataFeed, Exchange
from alpaca.trading.client import TradingClient
from alpaca.data import StockHistoricalDataClient
from alpaca.broker.client import BrokerClient

ImportError: cannot import name 'CorporateActionsRequest' from 'alpaca.trading.requests' (c:\Users\kalmar\.conda\envs\alpaca_env\lib\site-packages\alpaca\trading\requests.py)

In [4]:
alpaca = utils_for_alpaca.MyAlpaca(os.environ['ALPACA_API_KEY_PAPER'],os.environ['ALPACA_API_SECRET_PAPER'])

2025-01-20 23:02:48,699 - root - INFO - Successfully read config for PaperTesting strategy - [utils_for_alpaca.py:109]
2025-01-20 23:02:48,928 - root - INFO - Strategy PaperTesting has been initialized - [utils_for_alpaca.py:241]


In [7]:
API_KEY_PAPER = os.environ['ALPACA_API_KEY_PAPER']
API_SECRET_PAPER = os.environ['ALPACA_API_SECRET_PAPER']
stock_client = StockHistoricalDataClient(API_KEY_PAPER,API_SECRET_PAPER)
broker_client = BrokerClient(API_KEY_PAPER,API_SECRET_PAPER,sandbox=False,api_version="v2")
trading_client = TradingClient(API_KEY_PAPER, API_SECRET_PAPER) # dir(trading_client)

# Account information

### Data from account object

In [9]:
account = alpaca.trading_client.get_account()
#dir(account)
# Account properties description: https://docs.alpaca.markets/docs/account-plans#account-properties
account.__pretty__

<bound method Representation.__pretty__ of {   'account_blocked': False,
    'account_number': 'PA3BFSDVG64I',
    'accrued_fees': '0',
    'buying_power': '92497.76',
    'cash': '-114186.74',
    'created_at': datetime.datetime(2024, 5, 3, 20, 1, 7, 609557, tzinfo=TzInfo(UTC)),
    'crypto_status': <AccountStatus.ACTIVE: 'ACTIVE'>,
    'currency': 'USD',
    'daytrade_count': 0,
    'daytrading_buying_power': '92497.76',
    'equity': '106136.28',
    'id': UUID('dadd0de5-8668-43d1-8e56-1c7bde71b3bc'),
    'initial_margin': '121974.41',
    'last_equity': '104719.74',
    'last_maintenance_margin': '81595.3',
    'long_market_value': '220323.02',
    'maintenance_margin': '82634.97',
    'multiplier': '4',
    'non_marginable_buying_power': '0',
    'options_approved_level': 2,
    'options_buying_power': '23501.3',
    'options_trading_level': 2,
    'pattern_day_trader': True,
    'pending_transfer_in': None,
    'pending_transfer_out': None,
    'portfolio_value': '106136.28',
   

In [11]:
# Check our current balance vs. our balance at the last market close
balance_change = round(float(account.equity) - float(account.last_equity),2) 
balance_change

1416.54

In [12]:
account.buying_power # buying power is 2x equity if 2000 < equity < 25000, and 4x equity if equity > 25000
account.regt_buying_power # regulation T buying power = (2 x equity) - (long_position_value - short_position_value)

'0'

### Account activity types

In [18]:
# Here is smth wrong with limits
endpoint = '/account/activities'
params = {'after':'2023-11-30'}
pd.DataFrame(alpaca.trading_client.get(path=endpoint, data=params))

,id,activity_type,transaction_time,type,price,qty,side,symbol,leaves_qty,order_id,cum_qty,order_status,activity_sub_type,date,net_amount,description,status
0,20241220105727192::63ab8f03-0deb-4836-bd62-333...,FILL,2024-12-20T15:57:27.19223Z,fill,79.07,63,buy,PBH,0,13b56759-4723-4735-9fb8-effc19317ad3,74,filled,NaN,NaN,NaN,NaN,NaN
1,20241220105726781::63e587d5-05f5-49cc-9d9d-d35...,FILL,2024-12-20T15:57:26.781224Z,partial_fill,79.07,7,buy,PBH,63,13b56759-4723-4735-9fb8-effc19317ad3,11,partially_filled,NaN,NaN,NaN,NaN,NaN
2,20241220105726597::f23f0da9-0021-448e-94fc-891...,FILL,2024-12-20T15:57:26.597409Z,fill,79.07,79,buy,PBH,0,8686851d-4e61-478c-a66c-8d047f2d6b7f,79,filled,NaN,NaN,NaN,NaN,NaN
3,20241220105726539::172a9899-2093-4033-85b7-aa2...,FILL,2024-12-20T15:57:26.539478Z,fill,79.07,85,buy,PBH,0,c001d541-5a5c-43d8-846b-1da43da187d3,85,filled,NaN,NaN,NaN,NaN,NaN
4,20241220105726463::531307bf-bfa5-4249-916c-a81...,FILL,2024-12-20T15:57:26.463031Z,partial_fill,79.07,4,buy,PBH,70,13b56759-4723-4735-9fb8-effc19317ad3,4,partially_filled,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,20241217093029509::23f15f91-ec68-4383-a2ae-fc6...,FILL,2024-12-17T14:30:29.509455Z,partial_fill,31.86,10,buy,CNP,6,c4b3ebad-84a0-42c8-990f-cbbe74b51757,10,partially_filled,NaN,NaN,NaN,NaN,NaN
96,20241217093023348::09dce7ab-e953-498b-894c-c50...,FILL,2024-12-17T14:30:23.348259Z,fill,137.87,1,buy,LYV,0,1fe28d40-2048-48b0-9656-7977701875d6,3,filled,NaN,NaN,NaN,NaN,NaN
97,20241217093023095::9c6cdcc1-45c1-45b1-9352-c33...,FILL,2024-12-17T14:30:23.095575Z,partial_fill,137.87,2,buy,LYV,1,1fe28d40-2048-48b0-9656-7977701875d6,2,partially_filled,NaN,NaN,NaN,NaN,NaN
98,20241217000000000::d5049a34-99e5-4bc7-8623-92e...,FEE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,REG,2024-12-17,-3.55,REG fee for proceed of $127652.86 on 2024-12-1...,executed


In [15]:
activity = 'FEE'
endpoint = f'/account/activities/{activity}'
params = {'after':'2023-11-30','until':'2024-12-20'}
df_fees = pd.DataFrame(alpaca.trading_client.get(path=endpoint, data=params))
df_fees

,id,activity_type,activity_sub_type,date,net_amount,description,status
0,20241218000000000::d8d0cfa4-5263-45dc-922f-259...,FEE,TAF,2024-12-18,-0.15,TAF fee for proceed of 844 shares (8 trades) o...,executed
1,20241218000000000::280a7b24-9664-4eaf-acc6-e12...,FEE,REG,2024-12-18,-0.65,REG fee for proceed of $23354.98 on 2024-12-18...,executed
2,20241217000000000::d5049a34-99e5-4bc7-8623-92e...,FEE,REG,2024-12-17,-3.55,REG fee for proceed of $127652.86 on 2024-12-1...,executed
3,20241217000000000::31b1304e-b947-4935-8a2f-bd9...,FEE,TAF,2024-12-17,-0.26,TAF fee for proceed of 1535 shares (5 trades) ...,executed
4,20241216000000000::bce28164-5e3e-4805-8bb4-bfb...,FEE,REG,2024-12-16,-0.23,REG fee for proceed of $8257.82 on 2024-12-16 ...,executed
...,...,...,...,...,...,...,...
93,20240606000000000::00a672de-6c9b-476b-ad63-f9c...,FEE,NaN,2024-06-06,-0.81,TAF fee for proceed of 5561 shares (12 trades)...,executed
94,20240603000000000::85796647-02bd-4713-a377-241...,FEE,NaN,2024-06-03,-0.01,TAF fee for proceed of 14 shares (2 trades) on...,executed
95,20240603000000000::4cf843a8-de5e-4191-81af-5b2...,FEE,NaN,2024-06-03,-0.13,REG fee for proceed of $15879.55 on 2024-06-03...,executed
96,20240513000000000::3697788f-038f-4542-838f-8e3...,FEE,NaN,2024-05-13,-0.02,TAF fee for proceed of 100 shares (7 trades) o...,executed


In [ ]:
'FILL' # Order fills (both partial and full fills)
'TRANS' # Cash transactions (both CSD and CSW)
'MISC' # Miscellaneous or rarely used activity types (All types except those in TRANS, DIV, or FILL)
'ACATC' # ACATS IN/OUT (Cash)
'ACATS' # ACATS IN/OUT (Securities)
'CFEE' # Crypto fee
'CSD' # Cash deposit(+)
'CSW' # Cash withdrawal(-)
'DIV' # Dividends
'DIVCGL' # Dividend (capital gain long term)
'DIVCGS' # Dividend (capital gain short term)
'DIVFEE' # Dividend fee
'DIVFT'     # Dividend adjusted (Foreign Tax Withheld)
'DIVNRA'    # Dividend adjusted (NRA Withheld)
'DIVROC'    # Dividend return of capital
'DIVTW'    # Dividend adjusted (Tefra Withheld)
'DIVTXEX'    # Dividend (tax exempt)
'FEE'    # Fee denominated in USD
'INT'    # Interest (credit/margin)
'INTNRA'    # Interest adjusted (NRA Withheld)
'INTTW'    # Interest adjusted (Tefra Withheld)
'JNL'    # Journal entry
'JNLC'    # Journal entry (cash)
'JNLS'    # Journal entry (stock)
'MA'    # Merger/Acquisition
'NC'    # Name change
'OPASN'    # Option assignment
'OPEXP'    # Option expiration
'OPXRC'    # Option exercise
'PTC'    # Pass Thru Charge
'PTR'    # Pass Thru Rebate
'REORG'    # Reorg CA
'SC'    # Symbol change
'SSO'    # Stock spinoff
'SSP'    # Stock split

### Portfolio dynamic

In [ ]:
alpaca.trading_client.get_portfolio_history()

# Submit Orders

### Some Rules

In [ ]:
# Understanding orders: https://docs.alpaca.markets/docs/orders-at-alpaca
# Nice overview of different order types: https://alpaca.markets/learn/13-order-types-you-should-know-about/

# time-in-force options for various order types: https://docs.alpaca.markets/docs/orders-at-alpaca#order-types-vs-supported-time-in-force

# Trading is allowed from 4am till 8pm ET (10am - 2am Munich Time)

# Rules for submitting orders for extended hours: https://docs.alpaca.markets/docs/orders-at-alpaca#extended-hours-trading
    # extended hours order must be DAY limit orders: extended_hours=True & type='limit' & time_in_force=TimeInForce.DAY
        # A limit orders with a limit price that significantly exceeds the current market price will be rejected
        # Any other order types, including market orders, will be rejected

# Orders not eligible for extended hours submitted after 4:00pm ET will be queued up for release the next trading day.

# Your extended hour order will be processed and filled immediately.
        # not eligible orders: submitted between 4:00pm - 7:00pm ET will be rejected.
        # orders submitted after 7:00pm ET will be queued and eligible for execution at the time of the next market open

### Enums

In [ ]:
from alpaca.trading.enums import OrderSide, TimeInForce, OrderStatus, OrderType, OrderClass

In [33]:
classes = [TimeInForce, OrderSide, OrderClass, OrderType, OrderStatus]  # Replace with your actual classes
for cls in classes:
    filtered_members = [member for member in dir(cls) if not member.startswith('__')]
    print(f"{cls.__name__}:{filtered_members}")

TimeInForce:['CLS', 'DAY', 'FOK', 'GTC', 'IOC', 'OPG']
OrderSide:['BUY', 'SELL']
OrderClass:['BRACKET', 'OCO', 'OTO', 'SIMPLE']
OrderType:['LIMIT', 'MARKET', 'STOP', 'STOP_LIMIT', 'TRAILING_STOP']
OrderStatus:['ACCEPTED', 'ACCEPTED_FOR_BIDDING', 'CALCULATED', 'CANCELED', 'DONE_FOR_DAY', 'EXPIRED', 'FILLED', 'HELD', 'NEW', 'PARTIALLY_FILLED', 'PENDING_CANCEL', 'PENDING_NEW', 'PENDING_REPLACE', 'PENDING_REVIEW', 'REJECTED', 'REPLACED', 'STOPPED', 'SUSPENDED']


In [ ]:
# Time in force: https://docs.alpaca.markets/docs/orders-at-alpaca#time-in-force
    '''
    gtc - good-till-cancelled
            > aged order policy that will automatically cancel GTC orders 90 days
                > List/Get Orders API endpoint has a field titled expires_at, which provides information on the expiration
    day - eligible for execution only on the day it is live (9:30am - 4:00pm ET)
    opg - use it to submit “market on open” (MOO) and “limit on open” (LOO) orders: must be submitted after 7:00pm and before 9:28am
    cls - use it to submit “market on close” (MOC) and “limit on close” (LOC) orders
    ioc - Immediate Or Cancel (IOC) - all or part of the order to be executed immediately
    fok - Fill or Kill (FOK)
    '''


In [ ]:
# Bracket Order: https://docs.alpaca.markets/docs/orders-at-alpaca#bracket-orders

# Get Orders & Positions

### Order Statuses

In [7]:
from alpaca.trading.enums import OrderStatus

dir(OrderStatus)

# full list of statuses: https://alpaca.markets/docs/trading-on-alpaca/orders/#order-lifecycle

# Updates on orders states at Alpaca will be sent over the streaming interface


['ACCEPTED',
 'ACCEPTED_FOR_BIDDING',
 'CALCULATED',
 'CANCELED',
 'DONE_FOR_DAY',
 'EXPIRED',
 'FILLED',
 'HELD',
 'NEW',
 'PARTIALLY_FILLED',
 'PENDING_CANCEL',
 'PENDING_NEW',
 'PENDING_REPLACE',
 'PENDING_REVIEW',
 'REJECTED',
 'REPLACED',
 'STOPPED',
 'SUSPENDED',
 '__class__',
 '__doc__',
 '__members__',
 '__module__']

### Getting buy orders

In [12]:
from alpaca.trading.requests import GetOrdersRequest
from alpaca.trading.enums import QueryOrderStatus
date_filter = (pd.Timestamp.now()- pd.Timedelta(30, "days")) # orders for last 30 days
orders = alpaca.trading_client.get_orders(filter=GetOrdersRequest(status=QueryOrderStatus.CLOSED,after = date_filter))
bought_orders = [o for o in orders if o.side == "buy"]
bought_orders_df = pd.DataFrame(alpaca._orders_to_dictionary(bought_orders))
bought_orders_df['filled_avg_price'] = round(bought_orders_df['filled_avg_price'].astype(float),2)
bought_orders_df['filled_qty'] = bought_orders_df['filled_qty'].astype(int)
bought_orders_df['invest(k)'] = round((bought_orders_df['filled_qty'] * bought_orders_df['filled_avg_price'])/1000,1)
bought_orders_df['buytime'] = bought_orders_df['filled_at'].dt.tz_convert('Europe/Berlin').dt.tz_localize(None).dt.strftime('%d.%m %H:%M:%S')
bought_orders_df['strategy'] = bought_orders_df['client_order_id'].str.split('_').str[0]
columns_to_keep = ['strategy', 'symbol','invest(k)','buytime']
bought_orders_df = bought_orders_df[columns_to_keep]
bought_orders_df

,strategy,symbol,invest(k),buytime
0,OpenRangeBreakout,PBH,5.1,20.12 16:57:26
1,OpenRangeBreakout,PBH,5.2,20.12 16:57:26
2,OpenRangeBreakout,PBH,5.5,20.12 16:57:26
3,OpenRangeBreakout,PBH,5.9,20.12 16:57:27
4,OpenRangeBreakout,AVB,5.7,20.12 16:16:53
5,OpenRangeBreakout,PBH,6.2,20.12 16:57:26
6,OpenRangeBreakout,AVB,6.2,20.12 16:16:53
7,OpenRangeBreakout,PBH,6.7,20.12 16:57:26
8,OpenRangeBreakout,AVB,6.6,20.12 16:16:53
9,OpenRangeBreakout,WD,7.5,20.12 15:36:14


### Getting open orders

In [13]:
from alpaca.trading.requests import GetOrdersRequest
from alpaca.trading.enums import QueryOrderStatus
orders = alpaca.trading_client.get_orders(filter=GetOrdersRequest(status=QueryOrderStatus.OPEN))
open_buy_limit_orders = [o for o in orders if o.side == "buy" and o.order_type == "limit"]
open_sell_limit_orders = [o for o in orders if o.side == "sell" and o.order_type == "limit"]
open_other_orders = [o for o in orders if o.order_type != "limit"]
print(f'There are {len(open_buy_limit_orders)} open buy limit orders, {len(open_sell_limit_orders)} open sell limit orders and {len(open_other_orders)} other open orders.')

There are 0 open buy limit orders, 7 open sell limit orders and 0 other open orders.


### Get current positions

In [4]:
# details: https://docs.alpaca.markets/docs/position-average-entry-price-calculation#which-method-is-alpaca-using

positions = alpaca.trading_client.get_all_positions()
current_positions_df = pd.DataFrame(alpaca._orders_to_dictionary(positions))
cols_needed = ['symbol','market_value','unrealized_plpc']
current_positions_df = current_positions_df[cols_needed].copy()

current_positions_symbols = [p.symbol for p in positions]
buy_orders_dict = alpaca._get_orders_info(current_positions_symbols, datefilter=300)
buy_orders_df = pd.DataFrame.from_dict(buy_orders_dict, orient='index', columns=['filled_at', 'filled_avg_price', 'filled_qty', 'strat'])
buy_orders_df.reset_index(inplace=True)
buy_orders_df.rename(columns={'index': 'symbol'}, inplace=True)

current_positions_df = current_positions_df.merge(buy_orders_df, on='symbol', how='left')
current_positions_df['filled_at'] = current_positions_df['filled_at'].dt.tz_convert('Europe/Berlin').dt.tz_localize(None)
current_date = dt.datetime.now()
current_positions_df['days'] = (current_date - current_positions_df['filled_at']).dt.days
for column in ['market_value','unrealized_plpc']:
    current_positions_df[column] = pd.to_numeric(current_positions_df[column], errors='coerce')
current_positions_df['value'] = round(current_positions_df['market_value'] / 1000, 1)
current_positions_df['pl%'] = (current_positions_df['unrealized_plpc'] * 100).astype(int)
current_positions_df = current_positions_df.sort_values(by='pl%',ascending=False)

current_positions_df

2025-01-17 23:19:24,423 - root - INFO - Details (buytime, buyprice, qty, strategy) about bought positions (15) were extracted. - [utils_for_alpaca.py:986]


,symbol,market_value,unrealized_plpc,filled_at,filled_avg_price,filled_qty,strat,days,value,pl%
11,SPY,597.22,0.167745,2024-08-05 15:30:12.830152,511.43,1,c52eb28c-1294-460c-80c6-72b50c8ab4f8,165,0.6,16
10,SMAR,2145.86,0.156224,2024-07-23 16:30:25.924043,48.84,38,BuyStreaksAt11,178,2.1,15
13,VITL,7543.20,0.154835,2024-12-12 15:30:34.487996,38.88,168,BuyStreaksAt11,36,7.5,15
12,TPR,27128.20,0.095673,2024-12-16 16:19:30.080496,65.15,103,OpenRangeBreakout,32,27.1,9
2,ANDE,8125.20,0.088235,2024-12-20 15:40:52.276478,40.80,183,OpenRangeBreakout,28,8.1,8
5,BKU,1951.18,0.052048,2024-07-23 16:30:12.661888,37.85,49,BuyStreaksAt11,178,2.0,5
1,ALTR,79826.43,0.021936,2024-12-19 18:33:13.816129,108.04,64,OpenRangeBreakout,29,79.8,2
6,CALM,456.52,0.023312,2024-12-17 15:30:38.175639,111.53,4,BuyStreaksAt11,31,0.5,2
7,CNP,520.96,0.021971,2024-12-17 15:30:30.128961,31.86,16,BuyStreaksAt11,31,0.5,2
3,AVB,25740.52,-0.007552,2024-12-20 16:16:53.418603,219.80,34,OpenRangeBreakout,28,25.7,0


In [ ]:
trading_client.get_open_position('CALM').qty

### Time needed for update position

In [ ]:
            
# (1) First get the current position qty

symbol = 'SPY'
positions_list = trading_client.get_all_positions()
positions_df = pd.DataFrame([position for position in positions_list])

if positions_df.query('symbol==@symbol').empty:
    initial_qty = 0
else:
    initial_qty = positions_df.query('symbol==@symbol').qty.astype('int').iat[0]

# (2) place an order
market_order_data = MarketOrderRequest(symbol=symbol,qty=1,side=OrderSide.BUY,time_in_force=TimeInForce.DAY)
my_order = trading_client.submit_order(market_order_data)

# (3) Check time for filling
status = None
while status != 'filled':
    my_order = trading_client.get_order_by_id(my_order.get('id'))
    status = my_order.get('status')

created_at = pd.to_datetime(my_order.get('created_at')).tz_convert(None)
filled_at = pd.to_datetime(my_order.get('filled_at')).tz_convert(None)
current_time = pd.to_datetime('now', utc=True).tz_convert(None)

delta_fill_time = (current_time - created_at).total_seconds()
delta_poll_time = (current_time - filled_at).total_seconds()

print(f'{delta_fill_time} seconds to fill and {delta_poll_time} seconds polled after fill and status is {status}')

# (4) Check when in portoflio
filled_in_portfolio = False
while not filled_in_portfolio:
    positions_list = trading_client.get_all_positions()
    positions_df = pd.DataFrame([position for position in positions_list])

if positions_df.query('symbol==@symbol').empty:
    new_qty = 0
else:
    new_qty = positions_df.query('symbol==@symbol').qty.astype('int').iat[0]

if new_qty > initial_qty:
    filled_in_portfolio = True
    status = 'in portfolio'
else:
    filled_in_portfolio = False
    status = 'not in portfolio yet'    

current_time = pd.to_datetime('now', utc=True).tz_convert(None)
delta_time = (current_time - filled_at).total_seconds()
print(f'{delta_time} seconds after fill and status is {status}')


# Get Drawdowns

for every symbol in positions, check current drawdown and sell if critical


In [7]:
positions = alpaca.trading_client.get_all_positions()
current_positions_symbols = [p.symbol for p in positions]
buy_orders_dict = alpaca._get_orders_info(current_positions_symbols)

# Define a function to calculate drawdown
def calculate_drawdown(prices):
    max_prices = prices.cummax()
    drawdown = (prices - max_prices) / max_prices
    return drawdown

positions_dict_loss = {}
for symbol, details in buy_orders_dict.items():
    filled_at = details[0]
    data_df = alpaca.get_history_minute_single(symbol, filled_at)
    data_df['drawdown'] = data_df['close'].transform(calculate_drawdown)
    current_drawdown = data_df.iloc[-1]['drawdown'] # the rows are sorted from oldest to newest, so we take last row
    print(f'Current drawdown for {symbol} is {current_drawdown:.2%}')
    if current_drawdown < -alpaca.SimpleStopLoss:
        owned = int(alpaca.trading_client.get_open_position(symbol).qty)
        print(f'We need to sell {symbol} ({owned} stocks) as current drawdown ({current_drawdown:.2%}) bigger than allowed ({-alpaca.SimpleStopLoss:.2%})')

2024-11-08 14:49:23,102 - root - INFO - 152 rows were downloaded for TASK to get current drawdown. - [utils_for_alpaca.py:599]


Current drawdown for TASK is -0.51%


2024-11-08 14:49:23,351 - root - INFO - 149 rows were downloaded for UTHR to get current drawdown. - [utils_for_alpaca.py:599]


Current drawdown for UTHR is -0.69%


2024-11-08 14:49:23,595 - root - INFO - 157 rows were downloaded for DAY to get current drawdown. - [utils_for_alpaca.py:599]


...

Current drawdown for MCBS is -4.25%


2024-11-08 14:49:37,975 - root - INFO - 6150 rows were downloaded for CBOE to get current drawdown. - [utils_for_alpaca.py:599]


Current drawdown for CBOE is -7.48%
We need to sell CBOE (9 stocks) as current drawdown (-7.48%) bigger than allowed (-5.00%)


2024-11-08 14:49:39,547 - root - INFO - 6078 rows were downloaded for BKU to get current drawdown. - [utils_for_alpaca.py:599]


Current drawdown for BKU is -4.60%


# Assets in scope

### Asset enums

In [19]:
from alpaca.trading.enums import AssetClass, AssetStatus, AssetExchange
dir(AssetClass)

['CRYPTO',
 'US_EQUITY',
 'US_OPTION',
 '__class__',
 '__doc__',
 '__members__',
 '__module__']

In [20]:
dir(AssetStatus)

['ACTIVE', 'INACTIVE', '__class__', '__doc__', '__members__', '__module__']

In [21]:
dir(AssetExchange)

['AMEX',
 'ARCA',
 'BATS',
 'CBSE',
 'CRYPTO',
 'EMPTY',
 'ERSX',
 'FTXU',
 'GNSS',
 'NASDAQ',
 'NYSE',
 'NYSEARCA',
 'OTC',
 '__class__',
 '__doc__',
 '__members__',
 '__module__']

### Get ok stocks

In [24]:
def get_stocks():
    assets = alpaca.trading_client.get_all_assets(GetAssetsRequest(asset_class=AssetClass.US_EQUITY, status= AssetStatus.ACTIVE))
    exclude_strings = ['Etf', 'ETF', 'Lp', 'L.P', 'Fund', 'Trust', 'Depositary', 'Depository', 'Note', 'Reit', 'REIT']
    assets_in_scope = [asset.symbol for asset in assets
                        if asset.exchange != 'OTC' # OTC stocks play by different rules than Exchange Traded stocks (often referred to as NMS). 
                                                    # Data reporting is even different. 
                                                    # It’s not that one shouldn’t trade OTC stocks but be aware they have different rules. 
                                                    # Perhaps separate OTC stocks in your algo if you wish to trade these. 
                                                    # More about OTC: https://docs.alpaca.markets/docs/orders-at-alpaca#otc-positions
                        and asset.shortable
                        and asset.tradable
                        and asset.marginable # if a stock is not marginable that means it cannot be used as collateral for margin. 
                                                # As an example, if one had equity of $10,000 one could buy $10,000 of marginable stock and still have $10,000 left in buying power. 
                                                # However, if one were to buy $10,000 of non marginable stock the buying power would be $0 (ie no funds can be margined against that stock). 
                                                # This has ramifications for RegT buying power and Fed margin calls. Best to stay away from these unless your algo carefully monitors RegT buying power. 
                                                # FYI there are currently no tradable exchange traded stocks which are not marginable (so this doesn’t really limit the universe). However, all OTC stocks are non marginable (another reason to perhaps not consider OTC stocks)
                        and asset.fractionable # indirectly filters out a lot of small volatile stocks:  
                                                # Alpaca Trading Team manually reviews stocks to qualify as ‘fractionable’. 
                                                # This is entirely an Alpaca designation and other brokers may have different ‘fractionable’ stocks
                        and asset.easy_to_borrow # Alpaca currently uses its clearing firms ‘Easy To Borrow’ list and assumes everything else is Hard To Borrow. 
                        and asset.maintenance_margin_requirement == 30
                        and not (any(ex_string in asset.name for ex_string in exclude_strings))]
    return assets_in_scope
len(get_stocks())

2788

### Check spread of assets

In [26]:
assets = get_stocks()
latest_multisymbol_quotes = alpaca.stock_client.get_stock_latest_quote(StockLatestQuoteRequest(symbol_or_symbols=assets, feed=DataFeed.SIP))
rows = []
# TODO: check the bidask repo: https://github.com/eguidotti/bidask/tree/main/python
for symbol, info in latest_multisymbol_quotes.items():
    try:
        spread = (info.ask_price - info.bid_price) / info.ask_price
        rows.append({'symbol': symbol, 'spread': spread})
    except:
        continue
df_spreads = pd.DataFrame(rows)
df_spreads

,symbol,spread
0,HI,0.023005
1,ASTS,0.002868
2,JKHY,0.006103
3,LOVE,0.041589
4,ALIT,0.026087
...,...,...
2782,AME,0.007992
2783,CSV,0.478852
2784,TGT,0.002171
2785,CCJ,0.001161


### IPO Assets

In [4]:
assets = alpaca.trading_client.get_all_assets(GetAssetsRequest(asset_class=AssetClass.US_EQUITY, status= AssetStatus.ACTIVE))
# assets_in_scope = [asset.symbol for asset in assets if asset.exchange != 'OTC']
assets[1]

# https://docs.alpaca.markets/docs/orders-at-alpaca#ipo-symbols

# IS it only in broker client?

{   'asset_class': <AssetClass.US_EQUITY: 'us_equity'>,
    'attributes': [],
    'easy_to_borrow': False,
    'exchange': <AssetExchange.OTC: 'OTC'>,
    'fractionable': False,
    'id': UUID('27debfaf-b63f-40f5-a585-0423b96662ce'),
    'maintenance_margin_requirement': 100.0,
    'marginable': False,
    'min_order_size': None,
    'min_trade_increment': None,
    'name': 'NorthView Acquisition Corporation Red Wt Exp  08/02/2027',
    'price_increment': None,
    'shortable': False,
    'status': <AssetStatus.ACTIVE: 'active'>,
    'symbol': 'NVACW',
    'tradable': False}

# Strategies profitability

Orders for the last 90 days (max 500 orders)

In [4]:
days_of_stat = 90
date_filter = (pd.Timestamp.now()- pd.Timedelta(days_of_stat, "days")) # if it a stock in position, it has been bought with max_holding_time
request_params = GetOrdersRequest(status=QueryOrderStatus.CLOSED, after = date_filter, limit = 500)
orders_data = trading_client.get_orders(filter=request_params)
# some tech stuff as the orders_data are of strange type ----------------------------
orders_dicts = []
for entry in orders_data:
    entry_dict = {}  # Create a new dictionary for each entry
    for attribute, value in vars(entry).items():
        entry_dict[attribute] = value  # Populate the dictionary
    orders_dicts.append(entry_dict)  # Append the populated dictionary to the list
len(orders_dicts)

82

Getting current positions

In [ ]:
positions = trading_client.get_all_positions()
position_dicts = []
for entry in positions:
    entry_dict = {}  # Create a new dictionary for each entry
    for attribute, value in vars(entry).items():
        entry_dict[attribute] = value  # Populate the dictionary
    position_dicts.append(entry_dict)  # Append the populated dictionary to the list
current_positions = pd.DataFrame(position_dicts)
current_positions
cols_needed = ['symbol','qty','side','current_price']
current_positions = current_positions[cols_needed].copy()
current_positions['current_price'] = round(current_positions['current_price'].astype(float),2)
current_positions['qty'] = current_positions['qty'].astype(int)
current_positions

Orders as dataframe

In [ ]:
orders = pd.DataFrame(orders_dicts)
needed_cols = ['symbol','client_order_id', 'side','filled_at', 'filled_qty', 'filled_avg_price']
orders = orders[needed_cols].copy()
orders['filled_at'] = orders['filled_at'].dt.tz_convert('Europe/Berlin').dt.tz_localize(None)
# orders['filled_at'] = orders['filled_at'].dt.tz_convert('Europe/Berlin').dt.strftime('%Y-%m-%d %H:%M')
orders['filled_avg_price'] = round(orders['filled_avg_price'].astype(float),2)
orders['filled_qty'] = orders['filled_qty'].astype(int)
orders['strategy'] = orders['client_order_id'].str.split('_').str[0]
orders.loc[orders['strategy'].str[8] == '-', 'strategy'] = 'manual' # get rows where the 9th character in 'strategy' is "-" and update those rows to "manual"
# Sort the DataFrame by filled_at to ensure chronological order
orders.sort_values(by=['filled_at'], inplace=True)
orders

In [8]:
orders.to_excel('orders.xlsx')

In [7]:
from collections import deque

# Initialize dictionaries to hold the FIFO queue for each stock (long and short)
long_positions = {}
short_positions = {}

results = [] # Initialize a list to store the results

for index, row in orders.iterrows():
    symbol = row['symbol']
    side = row['side']
    filled_qty = row['filled_qty']
    filled_avg_price = row['filled_avg_price']
    filled_at = row['filled_at']
    strategy = row['strategy']
    
    if side == OrderSide.BUY:
        # Check if closing a short position
        if symbol in short_positions and short_positions[symbol]:
            qty_to_buy = filled_qty
            while qty_to_buy > 0 and short_positions[symbol]:
                sell_qty, sell_price, sell_filled_at, sell_strategy = short_positions[symbol].popleft()
                
                if sell_qty > qty_to_buy:
                    short_positions[symbol].appendleft((sell_qty - qty_to_buy, sell_price, sell_filled_at, sell_strategy))
                    results.append({
                        'symbol': symbol,
                        'filled_qty': qty_to_buy,
                        'buy_filled_at': filled_at,
                        'filled_avg_price_buy': filled_avg_price,
                        'filled_avg_price_sell': sell_price,
                        'profitability': (sell_price - filled_avg_price) * qty_to_buy,
                        'strategy': sell_strategy,
                        'days_in_portfolio': (filled_at - sell_filled_at).days,
                        'status': 'realized'
                    })
                    qty_to_buy = 0
                else:
                    results.append({
                        'symbol': symbol,
                        'filled_qty': sell_qty,
                        'buy_filled_at': filled_at,
                        'filled_avg_price_buy': filled_avg_price,
                        'filled_avg_price_sell': sell_price,
                        'profitability': (sell_price - filled_avg_price) * sell_qty,
                        'strategy': sell_strategy,
                        'days_in_portfolio': (filled_at - sell_filled_at).days,
                        'status': 'realized'
                    })
                    qty_to_buy -= sell_qty
        else:
            # Add the buy order to the FIFO queue for the stock
            if symbol not in long_positions:
                long_positions[symbol] = deque()
            long_positions[symbol].append((filled_qty, filled_avg_price, filled_at, strategy))

    elif side == OrderSide.SELL:
        # Check if closing a long position
        if symbol in long_positions and long_positions[symbol]:
            qty_to_sell = filled_qty
            while qty_to_sell > 0 and long_positions[symbol]:
                buy_qty, buy_price, buy_filled_at, buy_strategy = long_positions[symbol].popleft()
                
                if buy_qty > qty_to_sell:
                    long_positions[symbol].appendleft((buy_qty - qty_to_sell, buy_price, buy_filled_at, buy_strategy))
                    results.append({
                        'symbol': symbol,
                        'filled_qty': qty_to_sell,
                        'buy_filled_at': buy_filled_at,
                        'filled_avg_price_buy': buy_price,
                        'filled_avg_price_sell': filled_avg_price,
                        'profitability': (filled_avg_price - buy_price) * qty_to_sell,
                        'strategy': buy_strategy,
                        'days_in_portfolio': (filled_at - buy_filled_at).days,
                        'status': 'realized'
                    })
                    qty_to_sell = 0
                else:
                    results.append({
                        'symbol': symbol,
                        'filled_qty': buy_qty,
                        'buy_filled_at': buy_filled_at,
                        'filled_avg_price_buy': buy_price,
                        'filled_avg_price_sell': filled_avg_price,
                        'profitability': (filled_avg_price - buy_price) * buy_qty,
                        'strategy': buy_strategy,
                        'days_in_portfolio': (filled_at - buy_filled_at).days,
                        'status': 'realized'
                    })
                    qty_to_sell -= buy_qty            
        else:
            # Add the sell order to the FIFO queue for short positions
            if symbol not in short_positions:
                short_positions[symbol] = deque()
            short_positions[symbol].append((filled_qty, filled_avg_price, filled_at, strategy))

# Append remaining unsold stocks to results and use current_price for profitability calculation
for symbol, queue in long_positions.items():
    if symbol in current_positions['symbol'].values:
        current_price = current_positions[current_positions['symbol'] == symbol]['current_price'].values[0]
        while queue:
            buy_qty, buy_price, buy_filled_at, buy_strategy = queue.popleft()
            results.append({
                'symbol': symbol,
                'filled_qty': buy_qty,
                'buy_filled_at': buy_filled_at,
                'filled_avg_price_buy': buy_price,
                'filled_avg_price_sell': current_price,
                'profitability': (current_price - buy_price) * buy_qty,
                'strategy': buy_strategy,
                'days_in_portfolio': (dt.datetime.now() - buy_filled_at).days,
                'status': 'unrealized'
            })

for symbol, queue in short_positions.items():
    if symbol in current_positions['symbol'].values:
        current_price = current_positions[current_positions['symbol'] == symbol]['current_price'].values[0]
        while queue:
            sell_qty, sell_price, sell_filled_at, sell_strategy = queue.popleft()
            results.append({
                'symbol': symbol,
                'filled_qty': sell_qty,
                'buy_filled_at': dt.datetime.now(),
                'filled_avg_price_buy': current_price,
                'filled_avg_price_sell': sell_price,
                'profitability': (sell_price - current_price) * sell_qty,
                'strategy': sell_strategy,
                'days_in_portfolio': (dt.datetime.now() - sell_filled_at).days,
                'status': 'unrealized'
            })

# Convert results to DataFrame
result_df = pd.DataFrame(results)

In [ ]:
# result_df.to_excel('merged.xlsx')
result_df

In [17]:
result_df.groupby('strategy').agg(
    total_profit=('profitability', 'sum'),
    number_of_trades=('profitability', 'count'),
    average_profit=('profitability', 'mean')
).reset_index()

,strategy,total_profit,number_of_trades,average_profit
0,BuyStreaksAt11,34.07,44,0.774318
1,BuyTopPremarket,-1145.62,4,-286.405000
2,manual,3151.50,4,787.875000


# Update Relative Volume GH File

In my strat_config repo I have an csv file with average trade volumes for different stock. I use Openning-Range-Breakout strategy.

It is quicker to read the prepared file than to do recalculations. As I don't really need to do recalculation every time the strategy is run, I prefer to periodically (e.g. once a month) to update the csv file from here.

In [ ]:
import os
import utils_for_alpaca
alpaca = utils_for_alpaca.MyAlpaca(os.environ['ALPACA_API_KEY_PAPER'],os.environ['ALPACA_API_SECRET_PAPER'])
alpaca.update_database()

# LowPass Filter

LowPass (2-pole digital lowpass filter): suppresses high frequencies in the data series and thus attenuates all cycles that are shorter than the cutoff period. 

Compared to an EMA (which is in fact a 1-pole lowpass filter), cycles longer than the cutoff period are unaffected and passed without lag. This allows the LowPass function to react much faster and generate signals earlier than moving averages.

Replacing moving averages by lowpass filters improves most algorithms. MA is bad for trend finding. It lags a lotbehind the prices => the signals are just not timely for a good trade (the trend is already over when my moving average finally bends up or down)

Data

In [ ]:
from dateutil.relativedelta import relativedelta
clock = alpaca.trading_client.get_clock()
start_day = clock.timestamp.date() - relativedelta(days=10)
data_df = alpaca.get_history_minute_single('AAPL', start_day, window=10)
data_df

Formula

In [ ]:
Formula

# ATR for Stop-Loss

In [3]:
Universes.Spiders

['XLB', 'XLE', 'XLF', 'XLI', 'XLK', 'XLP', 'XLY', 'XLU', 'XLV']

In [12]:
days = 14
df_2hour = alpaca.get_history(symbols=Universes.Spiders,periods=3 * days, FrameLength=2, frame = 'hour')
df_2hour

2024-11-15 21:42:54,724 - root - INFO - Starting concurrent download of hour data across 9 tickers for 12 days ... - [utils_for_alpaca.py:481]
2024-11-15 21:42:56,322 - root - INFO - Done downloading hour data. 270 rows collected. - [utils_for_alpaca.py:493]


,symbol,timestamp,open,high,low,close,volume,trade_count,vwap
0,XLB,2024-11-04 11:00:00,93.93,93.950,93.115,93.390,1881240.0,19233.0,93.508072
1,XLB,2024-11-04 13:00:00,93.38,93.550,93.150,93.415,1423924.0,14994.0,93.429882
2,XLB,2024-11-04 15:00:00,93.42,93.550,93.330,93.540,1532957.0,11387.0,93.442497
3,XLB,2024-11-05 11:00:00,93.62,94.000,93.610,93.950,878936.0,11186.0,93.834754
4,XLB,2024-11-05 13:00:00,93.96,94.055,93.620,93.670,538998.0,7905.0,93.843818
...,...,...,...,...,...,...,...,...,...
265,XLY,2024-11-14 13:00:00,216.96,217.250,215.810,216.330,572407.0,10132.0,216.305522
266,XLY,2024-11-14 15:00:00,216.32,216.320,215.000,215.370,941038.0,14151.0,215.491792
267,XLY,2024-11-15 11:00:00,213.48,214.530,213.050,213.460,700871.0,11157.0,213.693828
268,XLY,2024-11-15 13:00:00,213.44,213.490,212.460,212.975,965155.0,13035.0,212.932340


In [13]:
# Calculate TR and ATR per symbol
def calculate_tr_and_atr(group, period=14):
    group['Prior Close'] = group['close'].shift() # Shift 'close' to get the previous close

    # Calculate True Range components
    group['High_Low'] = group['high'] - group['low']
    group['High_PriorClose'] = (group['high'] - group['Prior Close']).abs()
    group['Low_PriorClose'] = (group['low'] - group['Prior Close']).abs()   
    group['TR'] = group[['High_Low', 'High_PriorClose', 'Low_PriorClose']].max(axis=1)
    
    # for trasparency I included 4 steps above, but the 1 below could be used instead
    # group['TR'] = group[['high', 'Prior Close']].max(axis=1) - group[['low', 'Prior Close']].min(axis=1) # Calculate True Range (TR)

    group['ATR'] = group['TR'].rolling(window=period, min_periods=1).mean() # ATR as rolling mean of TR    
    return group

# Apply the function to each group of symbols
df_2hour = df_2hour.groupby('symbol').apply(calculate_tr_and_atr).reset_index(drop=True)
df_2hour

C:\Users\kalmar\AppData\Local\Temp\ipykernel_27176\211800909.py:18: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_2hour = df_2hour.groupby('symbol').apply(calculate_tr_and_atr).reset_index(drop=True)


,symbol,timestamp,open,high,low,close,volume,trade_count,vwap,Prior Close,High_Low,High_PriorClose,Low_PriorClose,TR,ATR
0,XLB,2024-11-04 11:00:00,93.93,93.950,93.115,93.390,1881240.0,19233.0,93.508072,NaN,0.835,NaN,NaN,0.835,0.835000
1,XLB,2024-11-04 13:00:00,93.38,93.550,93.150,93.415,1423924.0,14994.0,93.429882,93.390,0.400,0.160,0.240,0.400,0.617500
2,XLB,2024-11-04 15:00:00,93.42,93.550,93.330,93.540,1532957.0,11387.0,93.442497,93.415,0.220,0.135,0.085,0.220,0.485000
3,XLB,2024-11-05 11:00:00,93.62,94.000,93.610,93.950,878936.0,11186.0,93.834754,93.540,0.390,0.460,0.070,0.460,0.478750
4,XLB,2024-11-05 13:00:00,93.96,94.055,93.620,93.670,538998.0,7905.0,93.843818,93.950,0.435,0.105,0.330,0.435,0.470000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
265,XLY,2024-11-14 13:00:00,216.96,217.250,215.810,216.330,572407.0,10132.0,216.305522,217.000,1.440,0.250,1.190,1.440,2.041421
266,XLY,2024-11-14 15:00:00,216.32,216.320,215.000,215.370,941038.0,14151.0,215.491792,216.330,1.320,0.010,1.330,1.330,1.904993
267,XLY,2024-11-15 11:00:00,213.48,214.530,213.050,213.460,700871.0,11157.0,213.693828,215.370,1.480,0.840,2.320,2.320,1.984993
268,XLY,2024-11-15 13:00:00,213.44,213.490,212.460,212.975,965155.0,13035.0,212.932340,213.460,1.030,0.030,1.000,1.030,2.001421


In [16]:
current_atr_df = df_2hour.groupby('symbol').last().reset_index() # the rows are sorted from oldest to newest, so we take last row
current_atr_df['stop_loss_price'] = round(current_atr_df['close'] - 2 * current_atr_df['ATR'],2) # Calculate stop_loss_price as close - 2 * ATR
stop_loss_dict = current_atr_df.set_index('symbol')['stop_loss_price'].to_dict()
stop_loss_dict

{'XLB': 90.24,
 'XLE': 93.65,
 'XLF': 49.42,
 'XLI': 137.3,
 'XLK': 225.34,
 'XLP': 79.41,
 'XLU': 78.14,
 'XLV': 140.25,
 'XLY': 210.01}

# Open Range Breakout Strategy (with relative volumes)

In [2]:
import os
import utils_for_alpaca


In [ ]:
alpaca = utils_for_alpaca.MyAlpaca(os.environ['ALPACA_API_KEY_PAPER'],os.environ['ALPACA_API_SECRET_PAPER'])

In [ ]:
# getting latest volume data for currently-in-scope tickers 
all_tickers = alpaca.get_ok_alpaca_stocks(spread_limit = 0.015)
df_current = alpaca.get_history(symbols=all_tickers, periods=1, FrameLength=5, frame = 'min', only_for_today=True)
df_current

In [3]:
# reading current historical file with volumes from GH
from github import Github
import pandas as pd
repo = Github(os.getenv('github_strat_token')).get_user().get_repo(os.getenv('dedicated_repo'))        
history_volumes_df = pd.DataFrame()
history_volumes_df, sha = utils_for_alpaca._read_csv_from_github(repo, os.getenv('gh_csv_stocks_db'))
columns_to_convert = ['volume', 'trade_count']
history_volumes_df[columns_to_convert] = history_volumes_df[columns_to_convert].apply(pd.to_numeric, errors='coerce').astype(int)

2024-11-05 22:51:43,169 - root - INFO - Successfully read stocks_db.csv from GitHub - [utils_for_alpaca.py:89]


In [ ]:
# Adding historical average volumes to the df with current volumes
cols= ['symbol', 'volume', 'trade_count']
result_df = pd.merge(df_current[cols], history_volumes_df[cols], on=['symbol'], how='inner', suffixes=('_latest','_avg'))
result_df

In [ ]:
# calculating volume ratio selecting tickers with highest volume ratio
result_df['volume_ratio'] = result_df['volume_latest'] / result_df['volume_avg']
result_df['trade_count_ratio'] = result_df['trade_count_latest'] / result_df['trade_count_avg']
result_df = result_df.sort_values(by='volume_ratio', ascending=False)
ticker_rel_vol = result_df.head(alpaca.stocks_per_run).symbol.unique().tolist()
result_df

In [ ]:
# Getting today's high price for our filtered-by-volume tickers
# As I plan to run the strategy at 9:35 - it will be the high for the first 5min of trading
from alpaca.data.enums import DataFeed
from alpaca.data.requests import StockSnapshotRequest
snap = alpaca.stock_client.get_stock_snapshot(StockSnapshotRequest(symbol_or_symbols=ticker_rel_vol, feed = DataFeed.SIP))
snapshot_data = {stock: [snapshot.daily_bar.high, snapshot.daily_bar.low,]
                for stock, snapshot in snap.items() if snapshot and snapshot.daily_bar
                }
snapshot_columns=['today_high', 'today_low']
snapshot_df = pd.DataFrame(snapshot_data.values(), snapshot_data.keys(), columns=snapshot_columns)
snapshot_df

# Prices & Quotes

### Historical Prices

In [4]:
symbols = ['XLB', 'XLE', 'XLF', 'XLI', 'XLK', 'XLP', 'XLY', 'XLU', 'XLV']
df_day = alpaca.get_history(symbols=symbols, periods=1, FrameLength=1, frame = 'day')
df_day

2025-01-19 10:50:57,875 - root - INFO - Starting concurrent download of day data across 9 tickers for 3 days ... - [utils_for_alpaca.py:494]
2025-01-19 10:50:58,432 - root - INFO - Done downloading day data. 9 rows collected. - [utils_for_alpaca.py:508]


,symbol,timestamp,open,high,low,close,volume,trade_count,vwap
0,XLB,2025-01-17,88.34,88.900,88.0000,88.33,5794592.0,40521.0,88.483792
1,XLE,2025-01-17,93.24,94.290,92.9100,93.96,11949825.0,93948.0,93.925451
2,XLF,2025-01-17,49.91,50.320,49.7300,50.23,43693111.0,91335.0,50.144941
3,XLI,2025-01-17,137.81,138.265,137.2075,137.75,6869566.0,63163.0,137.791080
4,XLK,2025-01-17,234.66,234.940,233.0800,234.11,3371692.0,49555.0,234.184389
5,XLP,2025-01-17,76.94,77.400,76.8500,77.24,10525009.0,44529.0,77.214605
6,XLU,2025-01-17,78.68,79.200,78.4300,78.84,7334541.0,58414.0,78.886877
7,XLV,2025-01-17,140.94,141.400,140.1800,140.18,7668273.0,58035.0,140.821277
8,XLY,2025-01-17,229.03,230.660,228.6600,229.22,1942339.0,33310.0,229.713226


In [5]:
df_1min = alpaca.get_history(symbols=symbols, periods=5, FrameLength=1)
df_1min

2025-01-19 10:51:15,643 - root - INFO - Starting concurrent download of min data across 9 tickers for 3 days ... - [utils_for_alpaca.py:494]
2025-01-19 10:51:17,353 - root - INFO - Done downloading min data. 45 rows collected. - [utils_for_alpaca.py:508]


,symbol,timestamp,open,high,low,close,volume,trade_count,vwap
0,XLB,2025-01-17 16:00:00,88.3300,88.3500,88.3300,88.350,257543.0,5.0,88.330008
1,XLB,2025-01-17 15:59:00,88.3350,88.3700,88.3200,88.350,271770.0,1060.0,88.348370
2,XLB,2025-01-17 15:58:00,88.3800,88.3899,88.3200,88.340,203285.0,712.0,88.348816
3,XLB,2025-01-17 15:57:00,88.3650,88.3850,88.3650,88.375,67800.0,365.0,88.375193
4,XLB,2025-01-17 15:56:00,88.3600,88.3700,88.3500,88.370,39276.0,264.0,88.362489
5,XLE,2025-01-17 16:00:00,93.9600,93.9600,93.9000,93.900,416840.0,22.0,93.959561
6,XLE,2025-01-17 15:59:00,94.0100,94.0400,93.9600,93.960,364805.0,2170.0,93.999643
7,XLE,2025-01-17 15:58:00,94.0699,94.0800,94.0099,94.010,205618.0,1466.0,94.055749
8,XLE,2025-01-17 15:57:00,94.0400,94.0800,94.0300,94.065,93525.0,919.0,94.053905
9,XLE,2025-01-17 15:56:00,94.1050,94.1300,94.0200,94.040,99727.0,822.0,94.070408


In [6]:
df_15min = alpaca.get_history(symbols=symbols,periods=5)
df_15min

2025-01-19 10:53:09,802 - root - INFO - Starting concurrent download of min data across 9 tickers for 3 days ... - [utils_for_alpaca.py:494]
2025-01-19 10:53:10,308 - root - INFO - Done downloading min data. 45 rows collected. - [utils_for_alpaca.py:508]


,symbol,timestamp,open,high,low,close,volume,trade_count,vwap
0,XLB,2025-01-17 16:00:00,88.330,88.3500,88.3300,88.3300,258214.0,6.0,88.330008
1,XLB,2025-01-17 15:45:00,88.395,88.4180,88.3150,88.3500,1050040.0,5342.0,88.361868
2,XLB,2025-01-17 15:30:00,88.400,88.4600,88.3750,88.3900,274319.0,1876.0,88.409764
3,XLB,2025-01-17 15:15:00,88.365,88.4150,88.3300,88.4100,115383.0,862.0,88.368922
4,XLB,2025-01-17 15:00:00,88.420,88.4200,88.2900,88.3650,181229.0,1236.0,88.341328
5,XLE,2025-01-17 16:00:00,93.960,94.0003,93.9000,94.0000,430379.0,50.0,93.959495
6,XLE,2025-01-17 15:45:00,94.090,94.1600,93.9600,93.9600,1413811.0,10682.0,94.062843
7,XLE,2025-01-17 15:30:00,94.000,94.1100,93.9700,94.0850,559131.0,3968.0,94.047458
8,XLE,2025-01-17 15:15:00,93.995,94.0250,93.9350,94.0051,327502.0,2737.0,93.993719
9,XLE,2025-01-17 15:00:00,93.900,94.0100,93.8050,94.0000,358671.0,2601.0,93.892006


In [7]:
df_2hour = alpaca.get_history(symbols=symbols, periods=5, FrameLength=2, frame = 'hour')
df_2hour

2025-01-19 10:53:53,123 - root - INFO - Starting concurrent download of hour data across 9 tickers for 3 days ... - [utils_for_alpaca.py:494]
2025-01-19 10:53:53,288 - root - INFO - Done downloading hour data. 45 rows collected. - [utils_for_alpaca.py:508]


,symbol,timestamp,open,high,low,close,volume,trade_count,vwap
0,XLB,2025-01-17 15:00:00,88.420,88.530,88.2500,88.2500,1879386.0,9325.0,88.362478
1,XLB,2025-01-17 13:00:00,88.710,88.710,88.4000,88.4250,921668.0,6953.0,88.544645
2,XLB,2025-01-17 11:00:00,88.680,88.900,88.6000,88.7150,1365338.0,9985.0,88.748003
3,XLB,2025-01-16 15:00:00,87.560,87.770,87.4900,87.5600,1874277.0,11096.0,87.695127
4,XLB,2025-01-16 13:00:00,87.310,87.630,87.2750,87.5700,1189313.0,12269.0,87.420317
5,XLE,2025-01-17 15:00:00,93.900,94.160,93.8050,93.9800,3101831.0,20154.0,94.017246
6,XLE,2025-01-17 13:00:00,94.105,94.175,93.7600,93.9100,2419654.0,18879.0,93.964426
7,XLE,2025-01-17 11:00:00,94.030,94.290,93.8900,94.1050,2327809.0,20543.0,94.095618
8,XLE,2025-01-16 15:00:00,93.230,93.325,93.0500,93.1789,3274699.0,21633.0,93.204385
9,XLE,2025-01-16 13:00:00,92.940,93.270,92.9200,93.2400,2577975.0,20748.0,93.046406


### Snapshot

In [10]:
symbols = ['XLB', 'XLE', 'XLF', 'XLI', 'XLK', 'XLP', 'XLY', 'XLU', 'XLV']
snap = alpaca.stock_client.get_stock_snapshot(StockSnapshotRequest(symbol_or_symbols=symbols, feed = DataFeed.SIP))
snapshot_data = {stock: [
                        snapshot.latest_trade.timestamp,                        
                        snapshot.latest_trade.price, 
                        snapshot.daily_bar.open,
                        snapshot.daily_bar.close,
                        snapshot.previous_daily_bar.close,
                        ]
                for stock, snapshot in snap.items() if snapshot and snapshot.daily_bar and snapshot.previous_daily_bar
                }
snapshot_columns=['price time', 'price', 'today_open', 'today_close','yest_close']
snapshot_df = pd.DataFrame(snapshot_data.values(), snapshot_data.keys(), columns=snapshot_columns)
snapshot_df['price time'] = snapshot_df['price time'].dt.tz_convert('America/New_York').dt.tz_localize(None) # convert from UTC to ET and remove +00:00 from datetime
snapshot_df

,price time,price,today_open,today_close,yest_close
XLU,2025-01-17 20:00:00.006298,78.84,78.68,78.84,78.73
XLY,2025-01-17 20:00:00.003525,229.22,229.03,229.22,226.05
XLP,2025-01-17 20:00:00.004451,77.24,76.94,77.24,76.80
XLE,2025-01-17 20:00:00.006246,93.96,93.24,93.96,93.24
XLK,2025-01-17 20:00:00.006800,234.11,234.66,234.11,230.50
XLI,2025-01-17 20:00:00.002148,137.75,137.81,137.75,136.93
XLV,2025-01-17 20:00:00.001613,140.18,140.94,140.18,141.13
XLB,2025-01-17 20:00:00.001880,88.33,88.34,88.33,87.76
XLF,2025-01-17 20:00:00.006586,50.23,49.91,50.23,49.81


# Steps of a strategy

In [ ]:
alpaca = utils_for_alpaca.MyAlpaca(os.environ['ALPACA_API_KEY_PAPER'],os.environ['ALPACA_API_SECRET_PAPER'],'BuyTopPremarket')


In [ ]:

amount_to_invest = alpaca.strategy_buypower()

tickers_universe = alpaca.get_strategy_universe()
snapshot_df = alpaca.get_tkrs_snapshot_df(tickers_universe)
column_focus = 'POST'
tickers_to_buy_df = snapshot_df.sort_values(by=column_focus, ascending=False).head(alpaca.stocks_per_run)
tickers_to_buy_list = tickers_to_buy_df.symbol.unique().tolist()
tickers_to_buy_dict = utils_for_alpaca.create_dict_from_dataframe(tickers_to_buy_df,column_focus)
# investments = alpaca.allocate_capital(amount_to_invest, tickers=tickers_to_buy)
investments = alpaca.allocate_capital(amount_to_invest, method = 'manual', manual_weights=tickers_to_buy_dict)
alpaca.open_selected_positions(investments)

In [ ]:
alpaca = utils_for_alpaca.MyAlpaca(os.environ['ALPACA_API_KEY_PAPER'],os.environ['ALPACA_API_SECRET_PAPER'],'BuyStreaksAt11')
if alpaca.is_positive():
    amount_to_invest = alpaca.strategy_buypower()
    tickers_universe = alpaca.get_strategy_universe()
    df_day = alpaca.get_history(symbols=tickers_universe,periods=10, FrameLength=1, frame = 'day')
    df = alpaca.add_columns(df_day)
    latest_date = df['timestamp'].max()
    latest_date_rows = df[df['timestamp'] == latest_date].copy()
    max_streak = latest_date_rows['streak'].max()
    tickers_to_buy_prelim = latest_date_rows[latest_date_rows['streak'] == max_streak].symbol.unique().tolist()
    tickers_to_buy = alpaca.is_positive(tickers_to_buy_prelim) # 2nd filter - which are up today
    investments = alpaca.allocate_capital(amount_to_invest, tickers=tickers_to_buy)
    alpaca.open_selected_positions(investments)


# Time conversions

In [ ]:
snapshot_df['price time'] = snapshot_df['price time'].dt.tz_convert('America/New_York').dt.tz_localize(None) # convert from UTC to ET and remove +00:00 from datetime
snapshot_df['price time short'] = snapshot_df['price time'].dt.strftime('%H:%M')

now_time = SPY['SPY'].daily_bar.timestamp.strftime('%Y-%m-%d %H:%M:%S')

# Corporate Actions

In [ ]:
# https://docs.alpaca.markets/reference/corporateactions-1

In [2]:
from alpaca.data.enums import CorporateActionsType
from alpaca.data.requests import CorporateActionsRequest
from alpaca.data.historical.corporate_actions import CorporateActionsClient

API_KEY_PAPER = os.environ['ALPACA_API_KEY_PAPER']
API_SECRET_PAPER = os.environ['ALPACA_API_SECRET_PAPER']
corp_client = CorporateActionsClient(API_KEY_PAPER,API_SECRET_PAPER)

In [12]:
dir(CorporateActionsType)

['CASH_DIVIDEND',
 'CASH_MERGER',
 'FORWARD_SPLIT',
 'NAME_CHANGE',
 'REDEMPTION',
 'REVERSE_SPLIT',
 'RIGHTS_DISTRIBUTION',
 'SPIN_OFF',
 'STOCK_AND_CASH_MERGER',
 'STOCK_DIVIDEND',
 'STOCK_MERGER',
 'UNIT_SPLIT',
 'WORTHLESS_REMOVAL',
 '__class__',
 '__doc__',
 '__members__',
 '__module__']

In [13]:
# Today's actions
corp_request = CorporateActionsRequest()
corp_client.get_corporate_actions(corp_request)

{   'data': {   'cash_dividends': [   {   'corporate_action_type': 'cash_dividends',
                                          'due_bill_off_date': None,
                                          'due_bill_on_date': None,
                                          'ex_date': datetime.date(2024, 12, 31),
                                          'foreign': True,
                                          'payable_date': datetime.date(2025, 1, 20),
                                          'process_date': datetime.date(2025, 1, 20),
                                          'rate': 0.050485,
                                          'record_date': datetime.date(2024, 12, 31),
                                          'special': False,
                                          'symbol': 'BIRDF'},
                                      {   'corporate_action_type': 'cash_dividends',
                                          'due_bill_off_date': None,
                                          '

In [14]:
corp_request = CorporateActionsRequest(
    symbols=['NVDA'],
    start = '2023-09-11', # 90 days
    end = '2024-12-12', # 90 days
    limit=3000)
corp_client.get_corporate_actions(corp_request)

{   'data': {   'cash_dividends': [   {   'corporate_action_type': 'cash_dividends',
                                          'due_bill_off_date': None,
                                          'due_bill_on_date': None,
                                          'ex_date': datetime.date(2023, 9, 6),
                                          'foreign': False,
                                          'payable_date': datetime.date(2023, 9, 28),
                                          'process_date': datetime.date(2023, 9, 28),
                                          'rate': 0.04,
                                          'record_date': datetime.date(2023, 9, 7),
                                          'special': False,
                                          'symbol': 'NVDA'},
                                      {   'corporate_action_type': 'cash_dividends',
                                          'due_bill_off_date': None,
                                          'due_bill